## Import library

In [13]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import utils 
from tempfile import TemporaryFile

# save numpy array as npy file
from numpy import asarray
from numpy import save

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [14]:
tf.__version__

'2.2.0'

In [15]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

## Optimization with Adam and original function

In [4]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    delta = tf.Variable(100., trainable = True)
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(80000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [6]:

PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
losses = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    losses.append(loss.numpy())
    print("Data index {}. Delta {:.2f}. Loss {:.2f}".format(index_, delta_r, loss.numpy()))

SyntaxError: invalid syntax (<ipython-input-6-119b9b3d5526>, line 1)

In [ ]:
np.mean(losses)

In [ ]:
data = asarray(delta_seq)
save('delta.npy', delta_seq)

## Optimization with list of delta

In [16]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.05])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0.0, 
        stddev=1.0, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(40000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            ##print("Loss ", loss)
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

  1%|          | 1/100 [00:06<10:42,  6.49s/it]

The performance reach MAE: 0.001. Cancelling the training at step 16
Data index 1. Loss 0.04


  2%|▏         | 2/100 [00:12<10:24,  6.37s/it]

The performance reach MAE: 0.001. Cancelling the training at step 6
Data index 2. Loss 0.02


  3%|▎         | 3/100 [00:19<10:35,  6.55s/it]

...

The performance reach MAE: 0.001. Cancelling the training at step 110
Data index 27. Loss 0.05


 28%|██▊       | 28/100 [26:25<1:30:20, 75.29s/it] 

The performance reach MAE: 0.001. Cancelling the training at step 62
Data index 28. Loss 0.04


 29%|██▉       | 29/100 [26:31<1:04:27, 54.47s/it]

The performance reach MAE: 0.001. Cancelling the training at step 52
Data index 29. Loss 0.01


In [ ]:
save('list_delta_.npy', delta_seq)

## Optimization for individual treatment effect

In [11]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def optimization(data):
    threhold = tf.constant([0.01])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0, 
        stddev=10, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = data['mu1'] - data['mu0']
    
    for i in range(80000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            #print(true_effect)
            #print(influence)
            loss = tf.keras.losses.MAE(true_effect, influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            if i % 5000 == 0:
                print("Epoch {}. Loss {:.5f}".format(i, loss.numpy()))
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

In [12]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(13, 14)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0. Loss 175.66507
Epoch 5000. Loss 16.96419
Epoch 10000. Loss 11.77645
Epoch 15000. Loss 9.76440
Epoch 20000. Loss 8.73238
Epoch 25000. Loss 8.09316
Epoch 30000. Loss 7.64659
Epoch 35000. Loss 7.30521
Epoch 40000. Loss 7.03583
Epoch 45000. Loss 6.82551
Epoch 50000. Loss 6.65692
Epoch 55000. Loss 6.51818
Epoch 60000. Loss 6.40253
Epoch 65000. Loss 6.30351
Epoch 70000. Loss 6.21924
Epoch 75000. Loss 6.14970


100%|██████████| 1/1 [10:24<00:00, 624.99s/it]

Data index 13. Loss 6.09


In [12]:
save('individual_list_delta_.npy', delta_seq)